# Libraries





In [3]:
!pip install transformers==4.0.0
!pip install tensorboardX
!pip install pytorch-pretrained-bert
!pip install torch-optimizer


# Dataset

In [4]:
!wget https://github.com/facebookresearch/simmc/raw/master/data/simmc_fashion/fashion_train_dials.json

--2021-02-13 11:55:11--  https://github.com/facebookresearch/simmc/raw/master/data/simmc_fashion/fashion_train_dials.json
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/facebookresearch/simmc/master/data/simmc_fashion/fashion_train_dials.json [following]
--2021-02-13 11:55:11--  https://media.githubusercontent.com/media/facebookresearch/simmc/master/data/simmc_fashion/fashion_train_dials.json
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62887336 (60M) [application/octet-stream]
Saving to: ‘fashion_train_dials.json’

fashion_train_dials 100%[===================>]  59.97M   286

In [5]:
!wget https://github.com/facebookresearch/simmc/raw/master/data/simmc_fashion/fashion_dev_dials.json

--2021-02-13 11:55:12--  https://github.com/facebookresearch/simmc/raw/master/data/simmc_fashion/fashion_dev_dials.json
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/facebookresearch/simmc/master/data/simmc_fashion/fashion_dev_dials.json [following]
--2021-02-13 11:55:12--  https://media.githubusercontent.com/media/facebookresearch/simmc/master/data/simmc_fashion/fashion_dev_dials.json
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10390760 (9.9M) [application/octet-stream]
Saving to: ‘fashion_dev_dials.json’

fashion_dev_dials.j 100%[===================>]   9.91M  --.-KB/s   

In [6]:
!wget https://github.com/facebookresearch/simmc/raw/master/data/simmc_fashion/fashion_teststd_dials_public.json

--2021-02-13 11:55:13--  https://github.com/facebookresearch/simmc/raw/master/data/simmc_fashion/fashion_teststd_dials_public.json
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/facebookresearch/simmc/master/data/simmc_fashion/fashion_teststd_dials_public.json [following]
--2021-02-13 11:55:13--  https://media.githubusercontent.com/media/facebookresearch/simmc/master/data/simmc_fashion/fashion_teststd_dials_public.json
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3529872 (3.4M) [application/octet-stream]
Saving to: ‘fashion_teststd_dials_public.json’

fashion_teststd_dia 100%

In [7]:
!wget https://github.com/facebookresearch/simmc/raw/master/data/simmc_fashion/fashion_teststd_dials_api_calls.json

--2021-02-13 11:55:13--  https://github.com/facebookresearch/simmc/raw/master/data/simmc_fashion/fashion_teststd_dials_api_calls.json
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/facebookresearch/simmc/master/data/simmc_fashion/fashion_teststd_dials_api_calls.json [following]
--2021-02-13 11:55:13--  https://media.githubusercontent.com/media/facebookresearch/simmc/master/data/simmc_fashion/fashion_teststd_dials_api_calls.json
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 374581 (366K) [application/octet-stream]
Saving to: ‘fashion_teststd_dials_api_calls.json’

fashion_tests

# Imports and Dataset loading

In [8]:
import json
import warnings
import os
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import numpy as np

warnings.filterwarnings('ignore')


In [9]:
with open('fashion_train_dials.json') as json_file:
    data_train = json.load(json_file)

with open('fashion_dev_dials.json') as json_file:
    data_val = json.load(json_file)    

with open('fashion_teststd_dials_public.json') as json_file:
    data_test = json.load(json_file) 

with open('fashion_teststd_dials_api_calls.json') as json_file:
    data_test_gt = json.load(json_file)       

# Extraction

In [10]:
attributes = ['embellishment', 'availableSizes', 'price', 'info', 'customerRating', 'pattern', 'color', 'brand', 'size','material','clothingStyle',
              'clothingCategory', 'necklineStyle','hemLength','skirtStyle','dressStyle','jacketStyle']

In [11]:
def extract_info_attributes(round_datum):
    """Extract information attributes for current round using NLU annotations.
    Args:
        round_datum: Current round information
    Returns:
        get_attribute_matches: Information attributes
    """
    
    user_annotation = ast.literal_eval(round_datum["transcript_annotated"]) #round_datum["system_transcript_annotated"])
    
    annotation = user_annotation
    
    all_intents = [i["intent"] for i in annotation] 
    
    get_attribute_matches = [] # Information attributes
    
    for index, intent in enumerate(all_intents):
        
        if any(
            ii in intent
            for ii in ("DA:ASK:GET", "DA:ASK:CHECK", "DA:INFORM:GET")
        ):
              # If there is no attribute added, default to info. 
              if "." not in intent:
                   get_attribute_matches.append("info")
                   continue

              attribute = intent.split(".")[-1]
              if attribute == "info":
                  new_matches = [
                      ii["id"].split(".")[-1]
                      for ii in annotation[index]["slots"]
                      if "INFO" in ii["id"]
                  ]
                  if len(new_matches):
                          get_attribute_matches.extend(new_matches)
                  else:
                      get_attribute_matches.append("info")
              
              elif attribute != "":
                    get_attribute_matches.append(attribute)

    return sorted(set(get_attribute_matches)) 


In [12]:
def verifica(attr):
 attributes = ['embellishment', 'availableSizes', 'price', 'info', 'customerRating', 'pattern', 'color', 'brand', 'size','material','clothingStyle',
              'clothingCategory', 'necklineStyle','hemLength','skirtStyle','dressStyle','jacketStyle']
 
 for i,a in enumerate(attr):
    if a not in attributes:
      attr[i] = ""
 
 return attr      

In [13]:
def extract(data):
  task_mapping = {i["task_id"]: i for i in data["task_mapping"]} #creo dizionario con chiave taskid e intero
  
  dialogs = []                                                            # contenuto di task_mapping
  pattern = [] 

  for d in data['dialogue_data']:
    dialog_id = d["dialogue_idx"]
    if "dialogue_task_id" not in d:
        state = task_mapping[1874] #metto un task a caso per chi non ha dialogue task id(<0.1% dei dati)
    
    else:
        state = task_mapping[d["dialogue_task_id"]] #prendo task mapping del taskid del diaalogo
    
    roundwise_actions = []    
    
    for round_datum in d["dialogue"]:
        turn_idx = round_datum["turn_idx"]
        
        p = [dialog_id,turn_idx,round_datum['transcript'],'None',[]]

        insert_item = {
                "turn_idx": turn_idx,
                "action": "None",
                "action_supervision": None
            }

        keystrokes = round_datum.get("raw_assistant_keystrokes", []) #prendo keystrokes se non c'è lista vuota
        
        attributes = extract_info_attributes(round_datum)
        attributes = verifica(attributes)

        if keystrokes:
                focus_image = int(keystrokes[0]["image_id"]) #prendo focus image
                
                if focus_image in state["memory_images"]:
                    p = [dialog_id,turn_idx,round_datum['transcript'],'SearchMemory',attributes]
                    insert_item["action"] = "SearchMemory"
                    insert_item["action_supervision"] = {
                        "attributes": attributes,
                    }

                    
                elif focus_image in state["database_images"]:
                    p = [dialog_id,turn_idx,round_datum['transcript'],'SearchDatabase',attributes]
                    insert_item["action"] = "SearchDatabase"
                    insert_item["action_supervision"] = {
                        "attributes": attributes,
                    }
                #else:
                    #"Undefined action; using None instead"
                    
                pattern.append(p)
                roundwise_actions.append(insert_item)    
        else:
            
            if len(attributes):
                p = [dialog_id,turn_idx,round_datum['transcript'],'SpecifyInfo',attributes]
                insert_item["action"] = "SpecifyInfo"
                insert_item["action_supervision"] = {
                        "attributes": attributes
                    }
                
            else:
                # AddToCart action.
                #transcript annotated è stringa, ast.literal_eval la tratta come lista
                    for intent_info in ast.literal_eval(     
                        round_datum["transcript_annotated"]
                    ):
                        if "DA:REQUEST:ADD_TO_CART" in intent_info["intent"]:
                            p = [dialog_id,turn_idx,round_datum['transcript'],'AddToCart',[]] 
                            insert_item["action"] = "AddToCart"
                            insert_item["action_supervision"] = None   
                               
            roundwise_actions.append(insert_item)
            pattern.append(p)    
            
    dialogs.append(
            {
                "dialog_id": dialog_id,
                "actions": roundwise_actions,
            }
        )
    
  df = pd.DataFrame(pattern)
  df.columns = ['DialogID','TurnID','Text User','Action','Attributes']

  return df, dialogs

In [14]:
def extract_test(data):
  pattern = []
  for d in data['dialogue_data']:
    dialog_id = d["dialogue_idx"]
    
    for round_datum in d["dialogue"]:
        turn_idx = round_datum["turn_idx"]

        pattern.append((dialog_id,turn_idx,round_datum["transcript"]))
  
  
  df = pd.DataFrame(pattern)
  df.columns = ['DialogID','TurnID','Text User']

  return df    



In [15]:
df_train,dialogs_train = extract(data_train)
df_val,dialogs_val = extract(data_val)
df_test = extract_test(data_test)

# Processing

In [16]:
# compute the vocabulary and its size
import re
words = set()
for i in range(df_train.shape[0]):
    words = words.union(set(re.findall(r'\w+',df_train.iloc[i]['Text User'])))

vocab = dict(zip(list(words),range(1,len(words)+1)))

vocab['[UNK]'] = len(words)+1
vocab['[CLS]'] = len(words)+1
vocab['[SEP]'] = len(words)+1
vocab['[MASK]'] = len(words)+1
vocab['[PAD]'] = 0

with open('vocab.txt', 'w') as file:
     file.writelines([str(x)+'\n' for x in list(vocab.keys())]) 

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()
df_train['Attributes'] = list(mlb.fit_transform(df_train['Attributes']))
c=list(mlb.classes_) 
mlb = MultiLabelBinarizer(classes=c) 
df_val['Attributes'] = list(mlb.fit_transform(df_val['Attributes']))
d = list(mlb.classes_)

In [18]:
def frequenza(df):
  attr_freq = [0] * len(df['Attributes'][0])
  tot = 0
  for attr in df['Attributes']:
     i=0
     for a in attr:
        if (a==1):
          attr_freq[i] += 1
          tot+=1
        i+=1    
           
  return attr_freq, tot

In [19]:
freq_train,tot_freq_train = frequenza(df_train)
freq_val,tot_freq_val = frequenza(df_val)

In [20]:
freq_class = [freq/tot_freq_train for freq in freq_train]
bce_weights_train = torch.tensor([(1-freq)/freq for freq in freq_class])

In [21]:
def extract_humans(df):
  humans = list()
  for i in range(df.shape[0]):      
    human = df.iloc[i]['Text User']
    humans.append(human.lower())
  
  return humans  

In [22]:
humans_train = extract_humans(df_train)
humans_val = extract_humans(df_val)
humans_test = extract_humans(df_test)

# Training

In [23]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
from transformers import BertConfig, BertModel, BertTokenizer

tokenizer = BertTokenizer(vocab_file='vocab.txt')

def extract_encodeds(humans,df):

  decodeds = list()
  encodeds = list()

  for i in range(df.shape[0]):
    
    inputs_hum = tokenizer.tokenize(humans[i])
    encoded_sequence = tokenizer.encode(inputs_hum)
    decoded_sequence = tokenizer.decode(encoded_sequence)
    
    inputs = tokenizer(humans[i])
    encoded_sequence = inputs["input_ids"]
    
    decodeds.append(decoded_sequence)
    encodeds.append(encoded_sequence)
  
  return encodeds  

In [25]:
inputs_ids_train = extract_encodeds(humans_train,df_train)
inputs_ids_val = extract_encodeds(humans_val,df_val)
inputs_ids_test = extract_encodeds(humans_test,df_test)

In [26]:
def max_len(input_ids):
  max_len = 0

  for l in input_ids:
    if len(l) > max_len:
        max_len = len(l)
  
  return max_len      

In [27]:
max_len_train = max_len(inputs_ids_train)
max_len_val = max_len(inputs_ids_val)
max_len_test = max_len(inputs_ids_test)

In [28]:
def attention_masks_N_iid_padded(input_ids,max_len):
  attention_masks = list()
  iid_padded = list()
  tok_types_padded = list()

  for i in range(len(input_ids)):
    # produce the mask
    mask = [1]*len(input_ids[i]) + [0]*(max_len-len(input_ids[i]))
    attention_masks.append(mask)
    # pad encodeds and type current line
    # PAD = 0, since vocab starts from 1
    iid = input_ids[i] + [0]*(max_len-len(input_ids[i]))
    iid_padded.append(iid)
    # state to which sentence belong
    # and add PAD=0 at the end

  return attention_masks, iid_padded

In [29]:
attention_masks_train, iid_padded_train = attention_masks_N_iid_padded(inputs_ids_train,max_len_train)
attention_masks_val, iid_padded_val = attention_masks_N_iid_padded(inputs_ids_val,max_len_val)
attention_masks_test, iid_padded_test = attention_masks_N_iid_padded(inputs_ids_test,max_len_test)

In [30]:
df_train['Action'] = df_train['Action'].map({'SearchDatabase': 0, 'SearchMemory': 1,'SpecifyInfo' : 2,'AddToCart':3,'None':4})

df_val['Action'] = df_val['Action'].map({'SearchDatabase': 0, 'SearchMemory': 1,'SpecifyInfo' : 2,'AddToCart':3,'None':4})



In [31]:
attr_train = df_train['Attributes']

attr_val = df_val['Attributes']

In [32]:
actions_train = df_train['Action']

actions_val = df_val['Action']

In [33]:
dID_test = df_test['DialogID']
turn_test = df_test['TurnID']

In [34]:
import torch.nn as nn
from torch.utils.data import Dataset
import torch.utils.data
import torch.nn.functional as F

class Dataset(Dataset):
    
    def __init__(self, input_ids, attention_masks, attr, actions):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.attributes = attr
        self.actions = actions
        self.dataset_size = len(self.input_ids)
        
        
    def __getitem__(self,i):
        ids = torch.tensor(self.input_ids[i],dtype=torch.long)
        masks = torch.tensor(self.attention_masks[i],dtype=torch.long)
        attributes = torch.tensor(self.attributes[i],dtype=torch.float) 
        actions = torch.tensor(self.actions[i],dtype=torch.int64) 
            

        return ids, masks, attributes, actions
        
    def __len__(self):
        return self.dataset_size

In [35]:
class Dataset_test(Dataset):
    
    def __init__(self, input_ids, attention_masks,d_ID,turn):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.d_ID = d_ID
        self.turn = turn
        self.dataset_size = len(self.input_ids)
        
        
    def __getitem__(self,i):
        ids = torch.tensor(self.input_ids[i],dtype=torch.long)
        masks = torch.tensor(self.attention_masks[i],dtype=torch.long)
        ID = torch.tensor(self.d_ID[i],dtype=torch.int)
        turno = torch.tensor(self.turn[i],dtype=torch.int)

        return ids, masks, ID, turno
        
    def __len__(self):
        return self.dataset_size

In [36]:
train_loader = torch.utils.data.DataLoader(Dataset(iid_padded_train, 
                                                   attention_masks_train,
                                                   attr_train,
                                                   actions_train),
                                           batch_size=8,
                                           shuffle=True)

In [37]:
val_loader = torch.utils.data.DataLoader(Dataset(iid_padded_val, 
                                                   attention_masks_val,
                                                   attr_val,
                                                   actions_val),
                                           batch_size=8,
                                           shuffle=True)

In [38]:
test_loader = torch.utils.data.DataLoader(Dataset_test(iid_padded_test, 
                                                   attention_masks_test,
                                                   dID_test,
                                                   turn_test),
                                           batch_size=1,
                                           shuffle=True)

In [39]:
from transformers import BertConfig, BertModel, AdamW, BertForSequenceClassification


class MyBERTModel(nn.Module):
    def __init__(self):
          super(MyBERTModel, self).__init__()
          config = BertConfig(vocab_size=len(vocab))
          self.bert = BertModel(config=config)
          self.dropout = torch.nn.Dropout(0.3)
          #self.linear1 = torch.nn.Linear(768, 768)
          ### New layers:
          self.classifier1 = torch.nn.Linear(768, 5 ) ## we have 5 class
          self.classifier2 = torch.nn.Linear(768, 18) ## we have 34 labels in the output 
          
            
    def forward(self, ids, mask):
          sequence_output, pooled_output = self.bert(
               ids, 
               attention_mask = mask,
               return_dict=False)
         
            
         
          linear = self.dropout(pooled_output)
           
          classifier1_output = self.classifier1(linear)
          
          classifier2_output = self.classifier2(linear)
          
          return classifier1_output,classifier2_output

In [40]:
DEVICE = 'cuda'
model = MyBERTModel()
model = model.to(DEVICE)
LOG_FREQUENCY = 10
NUM_EPOCHS = 10

In [ ]:
from pytorch_pretrained_bert.optimization import BertAdam
from transformers import Adafactor
from torch_optimizer import Lamb
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
import copy
DEVICE = 'cuda'

criterion1 = torch.nn.CrossEntropyLoss().to(DEVICE)
criterion2 = torch.nn.BCEWithLogitsLoss(pos_weight=bce_weights_train).to(DEVICE)#

#optimizer = torch.optim.Adam(model.parameters(),lr=2e-5)
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay= 0.01, correct_bias=True) #1e-5

#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,8], gamma = 0.2)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * NUM_EPOCHS)

losses_train = {'total': [], 'actions': [], 'attributes': []}
losses_val = {'total': [], 'actions': [], 'attributes': []}

best_loss = 100

for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}'.format(epoch+1, NUM_EPOCHS))
  curr_epoch_losses = {'total': [], 'actions': [], 'attributes': []}

  model.train()

  current_step = 0
  
  for data in train_loader:

    in_ids = data[0].to(DEVICE) 
    att_mask = data[1].to(DEVICE) 
    attributes = data[2].to(DEVICE) 
    actions = data[3].to(DEVICE) 
     
    output1,output2 = model(in_ids,
                            att_mask)
              
    loss1 = criterion1(output1, actions)
    loss2 = criterion2(output2, attributes)
   
    
    
    total_loss = (loss1+loss2) / 2

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step() 
    
    curr_epoch_losses['total'].append(total_loss.item())
    curr_epoch_losses['actions'].append(loss1.item())
    curr_epoch_losses['attributes'].append(loss2.item())
    
    if current_step % (10*LOG_FREQUENCY) == 0:
      print('Step {}, Total Loss {}'.format(current_step, total_loss.item()))
      print('Step {}, Action Loss {}'.format(current_step, loss1.item()))
      print('Step {}, Attributes Loss {}\n\n'.format(current_step, loss2.item()))      
    
    current_step += 1

  #scheduler.step()    
  losses_train['total'].append(np.mean(curr_epoch_losses['total']))
  losses_train['actions'].append(np.mean(curr_epoch_losses['actions']))
  losses_train['attributes'].append(np.mean(curr_epoch_losses['attributes']))

  model.eval()
  curr_epoch_losses = {'total': [], 'actions': [], 'attributes': []}
  for data in val_loader:

    in_ids = data[0].to(DEVICE) 
    att_mask = data[1].to(DEVICE) 
    attributes = data[2].to(DEVICE) 
    actions = data[3].to(DEVICE) 
     
    output1,output2 = model(in_ids,
                            att_mask)
              
    loss1 = criterion1(output1, actions)
    loss2 = criterion2(output2, attributes)

    total_loss = (loss1+loss2) / 2

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step() 
    
    curr_epoch_losses['total'].append(total_loss.item())
    curr_epoch_losses['actions'].append(loss1.item())
    curr_epoch_losses['attributes'].append(loss2.item())
    

  losses_val['total'].append(np.mean(curr_epoch_losses['total']))
  losses_val['actions'].append(np.mean(curr_epoch_losses['actions']))
  losses_val['attributes'].append(np.mean(curr_epoch_losses['attributes'])) 

  if (losses_val['total'][-1] < best_loss):
    best_loss = losses_val['total'][-1]
    bestmodel = copy.deepcopy(model)
  
  scheduler.step()  

Starting epoch 1/10
Step 0, Total Loss 1.3411322832107544
Step 0, Action Loss 1.834660291671753
Step 0, Attributes Loss 0.8476042151451111




In [ ]:
torch.save(bestmodel.state_dict(), 'model_v20210207.pt')

In [ ]:
#epoch_count = range(1, NUM_EPOCHS + 1)
epoch_count = np.arange(1, NUM_EPOCHS+1)
plt.figure(figsize=(12,7))
plt.title("Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(epoch_count,losses_train['total'],color='red')
plt.plot(epoch_count,losses_train['actions'],color='blue')
plt.plot(epoch_count,losses_train['attributes'],color='green')
plt.legend(['Total Loss', 'Actions Loss','Attributes Loss'])
plt.show()


In [ ]:
epoch_count = np.arange(1, NUM_EPOCHS+1)
plt.figure(figsize=(12,7))
plt.title("Val loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(epoch_count,losses_val['total'],color='red')
plt.plot(epoch_count,losses_val['actions'],color='blue')
plt.plot(epoch_count,losses_val['attributes'],color='green')
plt.legend(['Total Loss', 'Actions Loss','Attributes Loss'])
plt.show()


In [ ]:
def ordina_attr(attributes_prediction):

  attributes = ['','availableSizes', 'brand', 'clothingCategory', 'clothingStyle', 'color', 'customerRating', 'dressStyle', 'embellishment', 'hemLength', 'info', 'jacketStyle', 'material', 'necklineStyle', 'pattern', 'price', 'size', 'skirtStyle']
  #attributes = ['ageRange', 'amountInStock', 'availableSizes', 'brand', 'clothingCategory', 'clothingStyle', 'color', 'customerRating', 'dressStyle', 'embellishment', 'forGender', 'forOccasion', 'hasPart', 'hemLength', 'hemStyle', 'info', 'jacketStyle', 'madeIn', 'material', 'necklineStyle', 'pattern', 'price', 'sequential', 'size', 'skirtLength', 'skirtStyle', 'sleeveLength', 'sleeveStyle', 'soldBy', 'sweaterStyle', 'waistStyle', 'warmthRating', 'waterResistance']
  a = []
  for i,attr in enumerate(attributes_prediction):
     if attr == 1:
       a.append(attributes[i])

  return a


In [ ]:
def ordina_action(action_prediction):

  actions=['SearchDatabase', 'SearchMemory','SpecifyInfo','AddToCart','None']

  a = actions[action_prediction]

  return a

In [ ]:
# Test
bestmodel = bestmodel.to(DEVICE)
bestmodel.eval()

actions_preds_dict = []
outputs = {}
actions=['SearchDatabase', 'SearchMemory','SpecifyInfo','AddToCart','None']

for data in test_loader:
    
    in_ids = data[0].to(DEVICE)
    att_mask = data[1].to(DEVICE)
    d_id = data[2].to(DEVICE)
    turn = data[3].to(DEVICE)
    
    output1, output2 = bestmodel(in_ids, att_mask)
    
    output1 = torch.nn.functional.softmax(output1,dim = 1)

    _,preds_action = torch.max(output1.data, dim=1)  
    
    preds_attributes = torch.sigmoid(output2)   # torch.Size([N, C]) e.g. tensor([[0., 0.5, 0.]])
    
    preds_attributes[preds_attributes >= 0.5] = 1
    preds_attributes[preds_attributes < 0.5] = 0
    
    actions_predictions = preds_action[0].item()
    actions_predictions = ordina_action(actions_predictions)
    
    attributes_predictions = {}
    attributes_predictions['attributes'] = ordina_attr(preds_attributes[0])
    
    action_log_prob = {}

    for i, prob in enumerate(output1.data[0]):
                action_log_prob[actions[i]] = torch.log(prob).item()

    actions_preds_dict.append( 
                  {
                    "dialog_id": d_id[0].item(),
                    "predictions": [
                        {
                            "action": actions_predictions,
                            "action_log_prob": action_log_prob,
                            "attributes": attributes_predictions,
                            "turn_id": turn[0].item()
                        }
                    ]
                }
            )      

In [ ]:
IGNORE_ATTRIBUTES = [
    "focus"  
]

def evaluate_action_prediction(
    gt_actions,
    model_actions,
    single_round_eval=False,
    compute_std_err=False,
    record_instance_results=None,
):
    """Evaluates action prediction using the raw data and model predictions.
    Args:
        gt_actions: Ground truth actions + action attributes
        model_actions: Actions + attributes predicted by the model
        single_round_eval: Evaluate only for the last turn
        compute_std_err: Computes standard error for the metrics
        record_instance_results: Record the result per instance
    """
    gt_actions_pool = {ii["dialog_id"]: ii for ii in gt_actions}
    matches = {"action": [], "attributes": [], "perplexity": []}
    confusion_dict = collections.defaultdict(list)
    for model_datum in model_actions:
        dialog_id = model_datum["dialog_id"]
        num_gt_rounds = len(gt_actions_pool[dialog_id]["actions"])
        for round_datum in model_datum["predictions"]:
            round_id = round_datum["turn_id"]
            # Skip if single_round_eval and this is not the last round.
            if single_round_eval and round_id != num_gt_rounds - 1:
                continue

            gt_datum = gt_actions_pool[dialog_id]["actions"][round_id]
            action_match = gt_datum["action"] == round_datum["action"]
            # Record matches and confusion.
            matches["action"].append(action_match)
            matches["perplexity"].append(
                round_datum["action_log_prob"][gt_datum["action"]]
            )
            confusion_dict[gt_datum["action"]].append(round_datum["action"])

            # Add the result to datum and save it back.
            if record_instance_results:
                round_datum["action_result"] = action_match
                round_datum["gt_action"] = gt_datum["action"]

            # Get supervision for action attributes.
            supervision = gt_datum["action_supervision"]
            if supervision is not None and "args" in supervision:
                supervision = supervision["args"]
            if supervision is None:
                continue
            # Case 1: Action mismatch -- record False for all attributes.
            if not action_match:
                for key in supervision.keys():
                    if key in IGNORE_ATTRIBUTES:
                        continue
                    matches["attributes"].append(False)
            # Case 2: Action matches -- use model predictions for attributes.
            else:
                for key in supervision.keys():
                    if key in IGNORE_ATTRIBUTES:
                        continue
                    gt_key_vals = supervision[key]
                    model_key_vals = round_datum["attributes"][key]
                    if not len(gt_key_vals):
                        continue
                    # For fashion, this is a list -- multi label prediction.
                    if isinstance(gt_key_vals, list):
                        assert isinstance(
                            model_key_vals, list
                        ), "Model should also predict a list for attributes"
                        recall = np.mean([(ii in model_key_vals) for ii in gt_key_vals])
                        if len(model_key_vals):
                            precision = np.mean(
                               [(ii in gt_key_vals) for ii in model_key_vals]
                            )
                        else:
                            precision = 0.0
                        f1_score = (2 * recall * precision) / (
                            recall + precision + 1e-5
                        )
                        matches["attributes"].append(f1_score)
                    else:
                        # For furniture, this is a string -- single label prediction.
                        matches["attributes"].append(gt_key_vals == model_key_vals)

    print("#Instances evaluated API: {}".format(len(matches["action"])))
    # Record and save per instance results.
    if record_instance_results:
        print("Saving per instance result: {}".format(record_instance_results))
        with open(record_instance_results, "w") as file_id:
            json.dump(model_actions, file_id)

    # Compute the confusion matrix.
    all_actions = sorted(
        set(confusion_dict.keys()).union(
            {jj for ii in confusion_dict.values() for jj in ii}
        )
    )
    matrix = np.zeros((len(all_actions), len(all_actions)))
    for index, action in enumerate(all_actions):
        labels, counts = np.unique(confusion_dict[action], return_counts=True)
        for label, count in zip(labels, counts):
            matrix[all_actions.index(label), index] += count

    metrics = {
        "action_accuracy": np.mean(matches["action"]),
        "action_perplexity": np.exp(-1 * np.mean(matches["perplexity"])),
        "attribute_accuracy": np.mean(matches["attributes"]),
        "confusion_matrix": matrix,
    }
    if compute_std_err:
        metrics_std_err = {
            "action_accuracy": (
                np.std(matches["action"]) / np.sqrt(len(matches["action"]))
            ),
            "action_perplexity": (
                (
                    np.exp(-1 * np.std(matches["perplexity"]))
                    / np.sqrt(len(matches["perplexity"]))
                )
            ),
            "attribute_accuracy": (
                np.std(matches["attributes"]) / np.sqrt(len(matches["attributes"]))
            ),
        }
        return metrics, metrics_std_err
    else:
        return metrics


# Metrics

In [ ]:
outputs = actions_preds_dict

In [ ]:
import collections
metrica = evaluate_action_prediction(data_test_gt,outputs)

In [ ]:
losses_train['actions']

In [ ]:
losses_train['attributes']

In [ ]:
losses_train['total']

In [ ]:
metrica